In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, Robot, bgr8_to_jpeg
import torchvision
import torch
import torchvision.transforms as transforms
import cv2
import PIL.Image
import os
import torch.nn.functional as F

In [2]:
camera = Camera.instance(width=224, height=224)

image_widget = widgets.Image(format='jpeg', width=224, height=224)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Collision models

In [26]:
model_col = torchvision.models.resnet18(pretrained=False)
model_col.fc = torch.nn.Linear(512, 2)
model_col.load_state_dict(torch.load('best_model_collision.pth'))
device = torch.device('cuda')
model_col = model_col.to(device)
model_col = model_col.eval()

### Forward model

In [25]:
model_forward = torchvision.models.resnet18(pretrained=False)
model_forward.fc = torch.nn.Linear(512, 1)
model_forward.load_state_dict(torch.load('best_steering_model_forward_merged.pth'))
device = torch.device('cuda')
model_forward = model_forward.to(device)
model_forward = model_forward.eval()

### Left model

In [4]:
model_left = torchvision.models.resnet18(pretrained=False)
model_left.fc = torch.nn.Linear(512, 1)
model_left.load_state_dict(torch.load('best_steering_model_left_merged.pth'))
device = torch.device('cuda')
model_left = model_left.to(device)
model_left = model_left.eval()

### Right model

In [5]:
model_right = torchvision.models.resnet18(pretrained=False)
model_right.fc = torch.nn.Linear(512, 1)
model_right.load_state_dict(torch.load('best_steering_model_right_merged.pth'))
device = torch.device('cuda')
model_right = model_right.to(device)
model_right = model_right.eval()

In [6]:
def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    #image = transforms.functional.to_grayscale(image, num_output_channels=3)
    #image = transforms.functional.resized_crop(image, crop_percent * 224, 0, 224 - crop_percent * 224, 224, (224, 224))
    image = transforms.functional.to_tensor(image)
    image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    return image[None, ...].to(device)

In [7]:
steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, value=0.0, description='steering')

display(steering_slider)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [8]:
speed_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.001, value=0.22, description='speed')
steering_bias = widgets.FloatSlider(min=-0.3, max=0.3, step=0.0001, value=0.0, description='bias')

display(speed_slider)
display(steering_bias)

FloatSlider(value=0.22, description='speed', max=1.0, step=0.001)

FloatSlider(value=0.0, description='bias', max=0.3, min=-0.3, step=0.0001)

In [9]:
from jetbot import Robot

robot = Robot()

In [10]:
kp = widgets.FloatSlider(min=0.0, max=1.0, step=0.001, value=0.4, description='kp')
kd = widgets.FloatSlider(min=0.0, max=1.0, step=0.001, value=0.37, description='kd')

display(kp, kd)

FloatSlider(value=0.4, description='kp', max=1.0, step=0.001)

FloatSlider(value=0.37, description='kd', max=1.0, step=0.001)

## Turn signal

In [19]:
robot.right_motor.alpha = -1.0
robot.left_motor.alpha = -1.0

In [11]:
turn_signal_map = {
    'forward': 0,
    'left': 1,
    'right': 2
}

turn_signal = widgets.ToggleButtons(
    options=['left', 'forward', 'right'],
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)
display(turn_signal)

ToggleButtons(options=('left', 'forward', 'right'), value='left')

## Blocks display

In [12]:
THUMBNAIL_DIR = 'thumbnails'

class_names = [
    'cross',
    'left',
    'right',
    'straight',
    't_left',
    't_right',
    't_straight'
]


# create thumbnail widgets
class_widgets = []
prob_widgets = []
for i, name in enumerate(class_names):
    
    # get thumbnail widget
    thumbnail_widget = widgets.Image(format='jpeg', width=80, height=80)
    thumbnail_path = os.path.join(THUMBNAIL_DIR, name + '.jpg')
    with open(thumbnail_path, 'rb') as f:
        thumbnail_widget.value = f.read()
    
    # get count widget
    prob_widget = widgets.FloatSlider(min=0.0, max=1.0, step=0.001, orientation='vertical')
    prob_widgets.append(prob_widget)
    
    class_widget = widgets.VBox([
        thumbnail_widget,
        prob_widget
    ])
    class_widgets.append(class_widget)

display(widgets.HBox(class_widgets))

In [13]:
transition_slider = widgets.FloatSlider(min=0.0, max=1.0, value=0.001, description='transition')
display(transition_slider)

FloatSlider(value=0.001, description='transition', max=1.0)

In [34]:
error = 0.0
error_last = 0.0

def execute(change):
    global error, error_last, error_i
    image = change['new']
    image = preprocess(image)
    
    output_col = F.softmax(model_col(image), dim=1).detach().cpu()
    
    # compute steering
    if turn_signal.value == 'left':
        output = model_left(image).detach().cpu()
    elif turn_signal.value == 'right':
        output = model_right(image).detach().cpu()
    else:
        output = model_forward(image).detach().cpu()
        
    steering = float(output[0][0]) + steering_bias.value
    steering_slider.value = steering
    
    # compute error, derivative and integral
    error = - steering
    error_d = error - error_last
    error_last = error
    
    pid_error = kp.value * error + kd.value * error_d
    
    speed = speed_slider.value
    
    if float(output_col.flatten()[0]) > 0.9:
        robot.stop()
    else:
        robot.set_motors(
            speed - pid_error,
            speed + pid_error
        )
    
execute({'new': camera.value})

In [35]:
camera.observe(execute, names='value')

In [36]:
import time

camera.unobserve_all()
time.sleep(0.5)
robot.stop()

In [ ]:
display(image_widget)
display(steering_slider)